In [ ]:
# Python 을 통해 Google API를 사용하기 위해서 라이브러리 다운로드
!pip install google-api-python-client
# Google 과 관련한 API를 사용하기 위해 필요한 모듈
!pip install google-auth-oauthlib google-auth-httplib2
# Youtube Captions 를 추출하기 위한 API를 사용하기 위해서 라이브러리 다운로드
!pip install youtube-transcript-api
!pip install nltk
!pip install vaderSentiment
!pip install spacy
!pip install textblob
!python -m spacy download en_core_web_sm
!pip install openai

In [ ]:
import openai
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import re
from datetime import datetime, timedelta

# OpenAI API 클라이언트 생성
openai.api_key = 'YOUR_KEY'  # 자신의 OpenAI API 키 입력

# YouTube API 클라이언트 생성
api_key = 'YOUR_KEY'  # 자신의 YouTube API 키 입력
youtube = build('youtube', 'v3', developerKey=api_key)

# spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# 감정 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

# 주제 관련 비디오 검색 함수
def search_videos_by_keyword(keyword, region_code='KR', max_results=3):
    numbers = [5, 10, 30, 50]  # 사용할 숫자 목록
    prefixes = ["Top", "Ranking"]  # 사용할 접두사 목록

    six_months_ago = datetime.now() - timedelta(days=180)
    published_after = six_months_ago.isoformat("T") + "Z"  # ISO 8601 형식으로 변환

    search_results = {}

    for number in numbers:
        for prefix in prefixes:
            query = f"{prefix} {number} {keyword} review"

            search_request = youtube.search().list(
                part='snippet',
                type='video',
                q=query,
                regionCode=region_code,
                maxResults=max_results,
                publishedAfter=published_after
            )

            search_response = search_request.execute()

            video_ids = [item['id']['videoId'] for item in search_response['items']]

            video_request = youtube.videos().list(
                part='snippet,statistics',
                id=','.join(video_ids)
            )

            video_response = video_request.execute()

            for item in video_response['items']:
                video_id = item['id']
                title = item['snippet']['title']
                view_count = int(item['statistics']['viewCount'])
                published_date = item['snippet']['publishedAt']

                try:
                    transcript = YouTubeTranscriptApi.get_transcript(video_id)
                    formatter = TextFormatter()
                    captions = formatter.format_transcript(transcript)
                    captions = split_into_sentences(captions)
                except Exception as e:
                    captions = f"No captions available or error occurred: {e}"

                search_results[video_id] = {
                    'title': title,
                    'views': view_count,
                    'published_date': published_date,
                    'captions': captions
                }

    return search_results

# 자막을 문장 단위로 분리하는 함수
def split_into_sentences(captions):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', captions)
    return " ".join(sentences)

# OpenAI GPT-4 모델을 사용하여 제품명 추출 함수
def extract_product_names_with_gpt(captions):
    try:
        response = openai.Completion.create(
            model="gpt-4o",
            prompt=f"Extract product names from the following text: {captions}",
            max_tokens=100,
            n=1,
            stop=None,
            temperature=0.5
        )
        product_names = response.choices[0].text.strip().split(",")
        product_names = [name.strip() for name in product_names]
        return product_names
    except Exception as e:
        print(f"Error in GPT-4o API call: {e}")
        return []

# 제품 리뷰 자막에서 주요 특징(Aspect) 추출 함수
def extract_aspects(captions, top_n=5):
    doc = nlp(captions)
    nouns = [chunk.text for chunk in doc.noun_chunks]

    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(nouns)
    tfidf_scores = dict(zip(vectorizer.get_feature_names_out(), X.sum(axis=0).tolist()[0]))
    sorted_tfidf = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)

    aspects = [aspect for aspect, score in sorted_tfidf[:top_n]]
    return aspects

# ABSA(Aspect-Based Sentiment Analysis) 함수
def analyze_sentiment(captions, aspects):
    aspect_sentiments = {}
    for aspect in aspects:
        sentences = [sent for sent in captions.split('. ') if aspect in sent]
        sentiment_scores = [analyzer.polarity_scores(sent)['compound'] for sent in sentences]
        avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
        aspect_sentiments[aspect] = avg_sentiment
    return aspect_sentiments

# 제품 자체의 감정 점수를 계산하는 함수 (특성의 감정 점수 기반)
def calculate_product_sentiment_from_aspects(sentiment_scores, weight=1.0):
    if sentiment_scores:
        avg_sentiment = (sum(sentiment_scores.values()) / len(sentiment_scores)) * weight
    else:
        avg_sentiment = 0
    return avg_sentiment

# 자막 데이터를 기반으로 제품명, 특징 및 감성 분석을 수행하는 함수
def process_single_caption(video_id, captions, weight=1.0):
    print(f"\nProcessing Video ID: {video_id}")
    product_names = extract_product_names_with_gpt(captions)

    if not product_names:
        print("No product names found.")
        return

    product_sentiments = {}

    print("Extracted product names:")
    for product in product_names:
        print(f"Product: {product}")

    for product in product_names:
        print(f"\nAnalyzing product: {product}")
        product_section = re.findall(f"{product}.*?(?=\\n[A-Z]|$)", captions, re.DOTALL)
        if product_section:
            product_section = product_section[0]
            aspects = extract_aspects(product_section)
            sentiment_scores = analyze_sentiment(product_section, aspects)
            product_sentiment = calculate_product_sentiment_from_aspects(sentiment_scores, weight)

            product_sentiments[product] = product_sentiment

            print(f"\nProduct Sentiment: {product_sentiment}")
            print("\nExtracted aspects and their sentiment scores:")
            for aspect, sentiment in sentiment_scores.items():
                print(f"Aspect: {aspect}, Sentiment: {sentiment}")
        else:
            print(f"No detailed section found for product: {product}")

    sorted_products = sort_products_by_sentiment(product_sentiments)
    print("\nProducts sorted by sentiment (descending):")
    for product, sentiment in sorted_products:
        print(f"Product: {product}, Sentiment: {sentiment}")

# 여러 동영상 자막을 처리하는 함수
def process_multiple_captions(captions_data, weights):
    final_results = []
    for video_id, data in captions_data.items():
        captions = data['captions']
        weight = weights.get(video_id, 1.0)
        process_single_caption(video_id, captions, weight)
        final_results.append((video_id, data['title'], weight))

    return final_results

# 가중치를 동적으로 계산하는 함수 (조회수 기반)
def calculate_dynamic_weights(captions_data):
    max_views = max(data['views'] for data in captions_data.values())
    weights = {}
    for video_id, data in captions_data.items():
        view_count = data['views']
        weights[video_id] = view_count / max_views
    return weights

# 제품 및 감정 점수 정렬 함수
def sort_products_by_sentiment(product_sentiments):
    return sorted(product_sentiments.items(), key=lambda x: x[1], reverse=True)

# 최종 결과 출력 및 정렬 함수
def print_final_results(final_results):
    sorted_results = sorted(final_results, key=lambda x: x[2], reverse=True)

    print("\nFinal Results (Sorted by Product Sentiment):")
    for video_id, title, sentiment in sorted_results:
        print(f"Video ID: {video_id}, Title: {title}, Sentiment: {sentiment}")

# 검색어로 비디오 검색 및 자막 데이터 가져오기
keyword = category # 'Mechanical Keyboard
captions_data = search_videos_by_keyword(keyword)

# 가중치 계산
weights = calculate_dynamic_weights(captions_data)

# 여러 자막 데이터 처리 후 최종 결과 출력
final_results = process_multiple_captions(captions_data, weights)
print_final_results(final_results)
